In [80]:
import pandas as pd 
import os
import xmltodict
import numpy as np
import xml.etree.ElementTree as etree
from collections import OrderedDict
#pd.options.display.max_rows  = 5000
pd.options.display.max_colwidth = -1
print("successfully imported")

successfully imported


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [276]:
tree = etree.parse("/Users/jamesz/dev/saraswati-data-quality/sample_CDA/C-CDA_R2-1_CCD.xml")
root = tree.getroot()

def iter_docs(author, elem):
    author_attr = author.attrib
    for doc in author.iter(elem):
        doc_dict = author_attr.copy()
        doc_dict.update(doc.attrib)
        doc_dict['data'] = doc.text
        yield doc_dict
        
def iter_author(etree):
    for author in etree.iter('author'):
        for row in iter_docs(author):
            yield row
            
def flatten_dict(d):
    def items():
        for key, value in d.items():
            if isinstance(value, dict):
                for subkey, subvalue in flatten_dict(value).items():
                    yield key + "." + subkey, subvalue
            else:
                yield key, value
    return OrderedDict(items())

def flatten_dict_detailed(d):
    def items():
        for key, value in d.items():
            if isinstance(value, dict):
                for i, (subkey, subvalue) in enumerate(flatten_dict_detailed1(value).items()):
                    yield key + f"{i}" + "." + subkey, subvalue
            elif isinstance(value, list):
                for i, val in enumerate(value):
                    if isinstance(val, str):
                        yield key + f'{i}', val
                    if isinstance(val, dict):
                        for subkey, subvalue in flatten_dict_detailed1(val).items():
                            yield key + f'{i}' + "." + subkey, subvalue
                    if isinstance(val, list):
                        yield flatten_dict_detailed(val)
            else:
                yield key, value
    
    return OrderedDict(items())

# Convert XML to dict
#TODO: create better CCD ingestion 
with open('/Users/jamesz/dev/saraswati-data-quality/sample_CDA/C-CDA_R2-1_CCD.xml', 'rb') as f:
    xml_content = xmltodict.parse(f)

# Flatten dict
flattened_xml = flatten_dict(xml_content)
agg = []

# Print in desired format
for k,v in flattened_xml.items():
    agg.append((k,v))
    
flattened_df = pd.DataFrame(agg, columns = ['tags', 'value'])


In [277]:
#list of sections in CCD
pd.DataFrame(list(iter_docs(root, '{urn:hl7-org:v3}title')))

,data
0,Patient Chart Summary
1,ADVANCE DIRECTIVES
2,ALLERGIES AND ADVERSE REACTIONS
3,ENCOUNTERS
4,FAMILY HISTORY
5,FUNCTIONAL STATUS
6,IMMUNIZATIONS
7,MEDICAL EQUIPMENT
8,MEDICATIONS
9,INSURANCE PROVIDERS


In [291]:
#@ = attribute
#. = next node
flattened_df.head(10)

,tags,value
0,ClinicalDocument.@xmlns,urn:hl7-org:v3
1,ClinicalDocument.@xmlns:xsi,http://www.w3.org/2001/XMLSchema-instance
2,ClinicalDocument.@xmlns:voc,urn:hl7-org:v3/voc
3,ClinicalDocument.@xmlns:sdtc,urn:hl7-org:sdtc
4,ClinicalDocument.realmCode.@code,US
5,ClinicalDocument.typeId.@extension,POCD_HD000040
6,ClinicalDocument.typeId.@root,2.16.840.1.113883.1.3
7,ClinicalDocument.templateId,"[{'@root': '2.16.840.1.113883.10.20.22.1.2', '@extension': '2015-08-01'}, {'@root': '2.16.840.1.113883.10.20.22.1.2'}]"
8,ClinicalDocument.id.@extension,TT988
9,ClinicalDocument.id.@root,2.16.840.1.113883.19.5.99999.1


In [279]:
#Find indexes where value is not string
nested_indexes = []
for i, val in enumerate(flattened_df.value):
    if type(val) == list:
        nested_indexes.append(i)

In [280]:
flattened_df.loc[nested_indexes].shape

(9, 2)

In [281]:
#flattened_df.loc[nested_indexes]

In [282]:
#hard coding for now
informant = pd.DataFrame(flattened_df.loc[131].value)
participant = pd.DataFrame(flattened_df.loc[186].value)
components = pd.DataFrame(flattened_df.loc[223].value)

In [283]:
#each row is another component (allergies, advance directives etc.)
components.shape

(15, 1)

In [284]:
#advanced directives section 
advance_directives = flatten_dict(components.section[0])
advance_directives_df = pd.DataFrame(advance_directives.items(), columns = ['tag', 'value'])

In [285]:
#immunizations section
Immunizations = flatten_dict(components.section[5])
Immunizations_df = pd.DataFrame(flatten_dict_detailed(Immunizations).items(), columns = ['tag', 'value'])

In [292]:
Immunizations_df.head(20)

,tag,value
0,templateId0.@root,2.16.840.1.113883.10.20.22.2.2.1
1,templateId0.@extension,2014-06-09
2,templateId1.@root,2.16.840.1.113883.10.20.22.2.2.1
3,code.@code,11369-6
4,code.@codeSystem,2.16.840.1.113883.6.1
5,code.@codeSystemName,LOINC
6,code.@displayName,History of immunizations
7,title,IMMUNIZATIONS
8,text.content.@ID,immunSect
9,text.table.@border,1


In [290]:
Immunizations_df[Immunizations_df.tag.str.contains('id[0-9].@root', regex=True)]

,tag,value
46,entry0.substanceAdministration6.id0.@root,e6f1ba43-c0ed-4b9b-9f12-f435d8ad8f92
66,entry0.substanceAdministration26.performer0.assignedEntity0.id0.@root,2.16.840.1.113883.19.5.9999.456
77,entry0.substanceAdministration37.performer11.assignedEntity11.representedOrganization0.id0.@root,2.16.840.1.113883.19.5.9999.1394
107,entry1.substanceAdministration6.id0.@root,e6f1ba43-c0ed-4b9b-9f12-f435d8ad8f92
127,entry1.substanceAdministration26.performer0.assignedEntity0.id0.@root,2.16.840.1.113883.19.5.9999.456
138,entry1.substanceAdministration37.performer11.assignedEntity11.representedOrganization0.id0.@root,2.16.840.1.113883.19.5.9999.1394
168,entry2.substanceAdministration6.id0.@root,e6f1ba43-c0ed-4b9b-9f12-f435d8ad8f92
188,entry2.substanceAdministration26.performer0.assignedEntity0.id0.@root,2.16.840.1.113883.19.5.9999.456
199,entry2.substanceAdministration37.performer11.assignedEntity11.representedOrganization0.id0.@root,2.16.840.1.113883.19.5.9999.1394
215,entry2.substanceAdministration53.entryRelationship7.observation5.id0.@root,4adc1020-7b14-11db-9fe1-0800200c9a64
